In [1]:
import gymnasium as gym
import yfinance as yf
import numpy as np
from stable_baselines3 import A2C, SAC, TD3
from stable_baselines3.common.logger import Logger
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.policies import ActorCriticPolicy

import gym_anytrading
from gym_anytrading.envs import StocksEnv
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import warnings
from classes.logger import NeptuneLogger, NeptuneCallback, TensorboardCallback
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
ticker = "AAPL"
stock_data_train = yf.Ticker(ticker).history(start="2020-01-01", end="2023-01-01")
stock_data_train = stock_data_train.drop(columns=["Dividends", "Stock Splits"])
stock_data_train

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02 00:00:00-05:00,72.151614,73.213531,71.895881,73.152641,135480400
2020-01-03 00:00:00-05:00,72.373271,73.208674,72.214960,72.441467,146322800
2020-01-06 00:00:00-05:00,71.554913,73.057662,71.301611,73.018692,118387200
2020-01-07 00:00:00-05:00,73.028440,73.286611,72.453647,72.675285,108872000
2020-01-08 00:00:00-05:00,72.375713,74.148816,72.375713,73.844368,132079200
...,...,...,...,...,...
2022-12-23 00:00:00-05:00,130.192282,131.683945,128.919399,131.127060,63814900
2022-12-27 00:00:00-05:00,130.649722,130.679555,128.004505,129.307220,69007800
2022-12-28 00:00:00-05:00,128.949229,130.301670,125.170356,125.339409,85438400


In [3]:
stock_data_train.dtypes

Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [4]:
make_kwargs_train={"df": stock_data_train,
            "frame_bound": (5,len(stock_data_train)),
            "window_size": 5,
            "available_capital": 10000}
gym.register(id="A2C-v0", entry_point='classes.stocks_env_box:StocksEnv')
env = gym.make('A2C-v0', apply_api_compatibility=False, **make_kwargs_train)
check_env(env)


In [5]:
env.prices.shape

(756,)

In [6]:
model = A2C("MlpPolicy", env, gamma=0.99, verbose=0, tensorboard_log="./a2c_stock_tensorboard/")
model.learn(total_timesteps=10000, progress_bar=True, log_interval=1, callback=TensorboardCallback(verbose=1))

Output()

In [7]:
%reload_ext tensorboard

In [8]:
%tensorboard --logdir ./a2c_stock_tensorboard/

In [9]:
np.iinfo(np.int32).max

2147483647

In [10]:
print(2**32 - 1)

4294967295


In [11]:
print(2**31-1)

2147483647
